In [5]:
import os, re, time, json
import pandas as pd
import requests
from requests.adapters import HTTPAdapter, Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any, Optional
from bs4 import BeautifulSoup

In [3]:
res = requests.get("https://web.getcha.kr/community/owner-review")

print(res.status_code)

soup = BeautifulSoup(res.text, "html.parser")

# 방법 1: script 태그에서 JSON 데이터 추출
scripts = soup.find_all("script")
print(f"총 script 태그 개수: {len(scripts)}")

# JSON 데이터가 들어있는 script 찾기
for i, script in enumerate(scripts):
    if script.string and "items" in script.string and "id_contents" in script.string:
        print(f"\n{i}번째 script에서 데이터 발견!")
        # JSON 파싱을 위해 일부 출력
        print(script.string[:500])





200
총 script 태그 개수: 90

89번째 script에서 데이터 발견!
self.__next_f.push([1,"7:[\"$\",\"$L27\",null,{\"data\":{\"page\":{\"currentPage\":1,\"totalPage\":228,\"totalContentsCount\":6837},\"items\":[{\"id_contents_migration\":154088,\"id_forum\":1,\"type\":11,\"id_contents\":8870,\"id_contents_topic\":2,\"contents_topic\":\"오너리뷰\",\"topic_tag\":null,\"title\":\"EV4\",\"contents\":\"혼자 출퇴근용으로 타는데 아주 만족입니다! 외관이 조금 이뻤으면 좋겠지만 전비가 좋으니 만족이에요! 트렁크랑 2열높이 개선만 되면 좋을 것 같아요 !! 매니저님께서 알아서 챙겨주시니 좋았습니다 내연\",\"reply_cnt\":2,\"like_cnt\":3,\"view_cnt\":67,\"is_notice


In [6]:
# 방법 2: 실제 JSON 데이터 추출 및 파싱

for script in scripts:
    if script.string and '"items":[{' in script.string:
        # self.__next_f.push([1, "..."]) 형태에서 JSON 부분만 추출
        script_text = script.string
        
        # 정규식으로 data 객체 찾기
        match = re.search(r'"data":\{.*?"items":\[(.*?)\]', script_text)
        if match:
            print("데이터 구조 발견!")
            
            # 전체 JSON 텍스트에서 items 배열 찾기
            # Next.js 데이터 구조 파싱
            try:
                # JSON 문자열에서 items 부분 추출
                items_match = re.search(r'"items":\[(\{.*?\})\]', script_text, re.DOTALL)
                if items_match:
                    print("\n첫 번째 아이템 샘플:")
                    print(items_match.group(1)[:1000])
            except Exception as e:
                print(f"파싱 오류: {e}")
            break


In [8]:
# 방법 3: HTML에서 Next.js 데이터 추출 (실제 작동하는 방법!)

# script 태그에서 데이터 추출
data_found = False

for script in scripts:
    if script.string and '"items":[{' in script.string:
        script_text = script.string
        
        # Next.js 데이터 형식: self.__next_f.push([1, "...JSON..."])
        # JSON 부분만 추출
        try:
            # "items":[ 로 시작하는 부분 찾기
            start_idx = script_text.find('"items":[{')
            if start_idx != -1:
                # 중첩된 배열과 객체를 고려하여 끝 찾기
                bracket_count = 0
                end_idx = start_idx + len('"items":[')
                
                for i in range(end_idx, len(script_text)):
                    if script_text[i] == '[':
                        bracket_count += 1
                    elif script_text[i] == ']':
                        bracket_count -= 1
                        if bracket_count == -1:  # items 배열의 끝
                            end_idx = i
                            break
                
                # JSON 문자열 추출
                json_str = '{' + script_text[start_idx:end_idx+1] + '}'
                
                # 이스케이프 처리된 문자열 정리
                json_str = json_str.replace('\\"', '"')
                json_str = json_str.replace('\\n', '\n')
                
                # JSON 파싱
                data = json.loads(json_str)
                data_found = True
                
                print("✅ 데이터 추출 성공!")
                print(f"총 리뷰 수: {len(data.get('items', []))}")
                
                # 첫 번째 리뷰 출력
                if data.get('items'):
                    first = data['items'][0]
                    print(f"\n첫 번째 리뷰:")
                    print(f"- 제목: {first.get('title')}")
                    print(f"- 작성자: {first.get('user', {}).get('nickname')}")
                    print(f"- 내용: {first.get('contents', '')[:100]}...")
                    print(f"- 좋아요: {first.get('like_cnt')}")
                
                break
                
        except Exception as e:
            print(f"파싱 중 오류: {e}")
            continue

if not data_found:
    print("❌ 데이터를 찾지 못했습니다.")


❌ 데이터를 찾지 못했습니다.


In [ ]:
# 방법 4: 리뷰 데이터를 DataFrame으로 변환

if data_found:
    reviews = []
    
    for item in data.get('items', []):
        review = {
            'id': item.get('id_contents'),
            '제목': item.get('title'),
            '내용': item.get('contents'),
            '작성자': item.get('user', {}).get('nickname'),
            '작성자레벨': item.get('user', {}).get('lv'),
            '좋아요': item.get('like_cnt'),
            '댓글수': item.get('reply_cnt'),
            '조회수': item.get('view_cnt'),
            '작성일': item.get('created'),
            '이미지수': item.get('image_count'),
            '이미지URL': item.get('image_url'),
            '평점': None  # tag_info에서 추출
        }
        
        # 평점 추출
        for tag in item.get('tag_info', []):
            if tag.get('type') == 'RATING':
                review['평점'] = float(tag.get('name'))
                break
        
        reviews.append(review)
    
    df = pd.DataFrame(reviews)
    print(f"\n총 {len(df)}개의 리뷰 수집")
    print(f"\n데이터 미리보기:")
    print(df[['제목', '작성자', '평점', '좋아요', '댓글수']].head(10))
    
    print(f"\n평점 통계:")
    print(df['평점'].describe())
